In [14]:
import warnings
from pylab import rcParams
from tensorflow_core.python.keras.wrappers.scikit_learn import KerasClassifier
rcParams['figure.figsize'] = 10, 5
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
import csv
from sklearn.metrics import accuracy_score
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [ ]:
data= pd.read_csv('../z_norm.csv')
### split data en X et Y
data1= data.copy()
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']

X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)

### Neural Network
## ANN Model
def create_model(optimizer):
    model = Sequential()
	#Layer 1
    model.add(Dense(20, activation='relu', input_dim=20))
    model.add(Dropout(0.3))
    #Layer 2
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.3))
    
    # output layer
    model.add(Dense(units=1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=50)
ANN = KerasClassifier(build_fn=create_model)

params={'optimizer' : ['adam', 'rmsprop'],
        'batch_size': [128, 256, 512, 800]}


##accuracy
grid_search_acc= GridSearchCV(estimator=ANN, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train, epochs=300, validation_data=(X_test, Y_test), callbacks=[early_stop])
y_predict= grid_search_acc.best_estimator_.predict(X_test)

Train on 71786 samples, validate on 30766 samples
Epoch 1/400
71786/71786 [==============================] - 3s 42us/sample - loss: 0.5683 - accuracy: 0.6675 - val_loss: 0.4317 - val_accuracy: 0.7994
Epoch 2/400
71786/71786 [==============================] - 2s 26us/sample - loss: 0.4284 - accuracy: 0.7905 - val_loss: 0.3716 - val_accuracy: 0.8292
Epoch 3/400
71786/71786 [==============================] - 2s 21us/sample - loss: 0.3919 - accuracy: 0.8234 - val_loss: 0.3607 - val_accuracy: 0.8319
Epoch 4/400
71786/71786 [==============================] - 2s 22us/sample - loss: 0.3768 - accuracy: 0.8291 - val_loss: 0.3584 - val_accuracy: 0.8324
Epoch 5/400
71786/71786 [==============================] - 2s 25us/sample - loss: 0.3742 - accuracy: 0.8325 - val_loss: 0.3563 - val_accuracy: 0.8334
Epoch 6/400
71786/71786 [==============================] - 2s 28us/sample - loss: 0.3708 - accuracy: 0.8333 - val_loss: 0.3553 - val_accuracy: 0.8337
Epoch 7/400
71786/71786 [=========================

Epoch 55/400
71786/71786 [==============================] - 2s 27us/sample - loss: 0.3590 - accuracy: 0.8349 - val_loss: 0.3523 - val_accuracy: 0.8351
Epoch 56/400
71786/71786 [==============================] - 2s 35us/sample - loss: 0.3569 - accuracy: 0.8358 - val_loss: 0.3520 - val_accuracy: 0.8360
Epoch 57/400
71786/71786 [==============================] - 2s 30us/sample - loss: 0.3579 - accuracy: 0.8355 - val_loss: 0.3521 - val_accuracy: 0.8351
Epoch 58/400
71786/71786 [==============================] - 2s 33us/sample - loss: 0.3580 - accuracy: 0.8354 - val_loss: 0.3519 - val_accuracy: 0.8353
Epoch 59/400
71786/71786 [==============================] - 2s 27us/sample - loss: 0.3571 - accuracy: 0.8353 - val_loss: 0.3517 - val_accuracy: 0.8347
Epoch 60/400
71786/71786 [==============================] - 2s 23us/sample - loss: 0.3582 - accuracy: 0.8360 - val_loss: 0.3523 - val_accuracy: 0.8350
Epoch 61/400
71786/71786 [==============================] - 2s 25us/sample - loss: 0.3581 - ac

In [ ]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'ANN', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [22]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")

----------
best parameters:  {'batch_size': 256, 'optimizer': 'rmsprop'}
----------
[[11245  4114]
 [  940 14467]]
----------
              precision    recall  f1-score   support

           0       0.92      0.73      0.82     15359
           1       0.78      0.94      0.85     15407

    accuracy                           0.84     30766
   macro avg       0.85      0.84      0.83     30766
weighted avg       0.85      0.84      0.83     30766

----------
